In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import exp


An instant hit response (can be SPE or multiple photons)

In [2]:
def peResponse(t, delay, nphotons, speAmplitude, rise_time, fall_time):
    if t < delay:
        return 0
    else:
        return -nphotons * speAmplitude * exp(-(t-delay) / fall_time) * (1 - exp(-(t-delay) / rise_time))


Generate waveform with multiple hits at random time

In [3]:
def waveGen(t, speAmplitude, rise_time, fall_time):
    response = np.array([0 for ti in t])
    
    while True:
        nhits = 0
        nhits = np.random.poisson(2)
        if nhits > 0: 
            break
    
    while True:
        checkSumHits = 0
        for aHit in range(0, nhits):
            delay = np.random.uniform(t[0], t[-1])
            nphotons = np.random.poisson(3)
            checkSumHits = checkSumHits + nphotons
            tempResponse = np.array([(peResponse(ti, delay, nphotons, speAmplitude, rise_time, fall_time) \
                                      + np.random.normal(0, speAmplitude / 15)) for ti in t])
            response = response + tempResponse
        if checkSumHits > 0:
            break
            
    return response

Digitizing the waveform

In [4]:
def getADC(pi, nBits, res, voltMin):
    guessADC =  (pi - voltMin) / res
    if guessADC < 0:
        return 0
    if guessADC > (2**nBits - 1):
        return 2**nBits - 1
    return guessADC

In [5]:
def digitizeWave(p, nBits, voltMin, dynamicRange, offset):
    resolution = dynamicRange / (2**nBits - 1)
    return np.array([(getADC(pi, nBits=nBits, res=resolution, voltMin=voltMin) + offset) for pi in p]) 

A triggered waveform

In [6]:
def aTrigger(nBits, voltMin, dynamicRange, offset):
    t = np.arange(0, 204.8, 0.2)
    p = waveGen(t, 0.14, 1, 3)
    digital_p = digitizeWave(p, nBits=nBits, voltMin=voltMin, dynamicRange=dynamicRange, offset=offset)
    
    return [t, p]

A test: generate a triggered signal from SiPM

In [14]:
[t, p] = aTrigger(12, -0.1, 1, 1000)
plt.plot(t, p)
plt.show()